In [629]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
purchase_df = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_df = pd.read_csv(purchase_df)

In [630]:
purchase_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [631]:
#Get the total number of players
total_players = purchase_df["SN"].nunique()

#creating a dataframe summary table for total_players
total_players_df = pd.DataFrame({"Total Players":[total_players]})
total_players_df


,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [632]:
#unique items - Number of Unique Items
unique_items = purchase_df["Item ID"].nunique()

#average price - Average Price
average_price = purchase_df["Price"].mean()

# total_number of purchases - Number of Purchases
total_num_purchases = purchase_df["Purchase ID"].count()

# total purchase -Total Revenue
total_revenue = purchase_df["Price"].sum() 

# Creating a summary-table to hold the results
purchase_analysis_total = pd.DataFrame({"Number of Unique Items":[unique_items],
                                           "Average Price": [average_price],
                                           "Number of Purchases":[total_num_purchases],
                                           "Total Revenue":[total_revenue]})

# formated data table with currency
purchase_analysis_total["Average Price"] = purchase_analysis_total["Average Price"].map("${:.2f}".format)
purchase_analysis_total["Total Revenue"] = purchase_analysis_total["Total Revenue"].map("${:,.2f}".format)


# Display the summary DataFrame 
purchase_analysis_total

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

In [633]:
#run .dtypes to see data types
purchase_df.dtypes

Purchase ID      int64
SN              object
Age              int64
Gender          object
Item ID          int64
Item Name       object
Price          float64
dtype: object

In [634]:
# Identifyin duplicate SN which are the Screen Names; 
# therefore, we are not calculating them twice for our count
purchase_df.SN.duplicated().sum()

204

In [635]:
# Dropping duplicates for SN
drop_dups = purchase_df.drop_duplicates(["SN"], keep ="last")


In [636]:
# Counting number of each gender after duplicates were dropped
gender_df = purchase_df.groupby(["Gender"])

gender_count = gender_df["SN"].nunique()

# percentage of players by each gender, values are rounded by 2 decimal points and added % sign
gender_percentage = round((gender_count / total_players ) *100,2).map("{:,}%".format)

# Create data frame with obtained values
gender_summary =pd.DataFrame({"Total Count":gender_count, "Percentage of Players": gender_percentage})

# Sorting values from the highest count and Percentage of Players to lowest
gender_summary_df = gender_summary.sort_values(["Total Count", "Percentage of Players"], ascending = False)

# Highlighting Highest and lowest value different colors
gender_summary_df = (gender_summary_df.style.highlight_max(color='lime').highlight_min(color='lightcoral'))

gender_summary_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [652]:
# Columns: Gender, Purchase Count, Average Purchase Price, Total Purchase Value, Avgerage Total Purchase per Person?

# Using GroupBy in order to separate the data into fields according to "Gender" values

# get gender count from purchase dataframe

purchase_count = gender_df["Purchase ID"].count()

# Find average count by each group

# Find total purchase value for each gender
total_purchase = gender_df["Price"].sum()

# Calculating Average Purchase price per Gender
average_purchase_price = round((total_purchase/purchase_count),2)


# Find average purchase total per person by gender; by dividing each genders total purchases to unique gender count
average_purchase_total = round((total_purchase / gender_count),2)


# Create a data frame to display results from the calculations
purchase_summary = pd.DataFrame({"Purchase Count":purchase_count, 
                                "Average Purchase Price":average_purchase_price, 
                                "Total Purchase Value": total_purchase,
                                "AVG Total Purchase per Person":average_purchase_total})

# Format average purchase price, total purchase values and average total purchase values

purchase_summary["Average Purchase Price"] = purchase_summary["Average Purchase Price"].map("${:.2f}".format)

purchase_summary["Total Purchase Value"] = purchase_summary["Total Purchase Value"].map("${:.2f}".format)

purchase_summary["AVG Total Purchase per Person"] = purchase_summary["AVG Total Purchase per Person"].map("${:.2f}".format)

# I could use Sorting values from the highest count and Percentage of Players to lowest to sort out data
#purchase_summary = purchase_summary.sort_values(["Purchase Count", "Average Purchase Price", "Total Purchase Value", "AVG Total Purchase per Person"], ascending = False)

# Used .sort_index() to sort my series by index labels, used this method since it's a shorter coding line
purchase_summary = purchase_summary.sort_index()
purchase_summary

,Purchase Count,Average Purchase Price,Total Purchase Value,AVG Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [638]:
# Figure out the minimum and maximum Age for purchase data to establish the bin
print(purchase_df["Age"].max())
print(purchase_df["Age"].min())

45
7


In [639]:
#Establish bins for ages
 
bins = [5,9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 99]

# Create bins and label lists acording to min age 7 and max age 45 to hold age data accordingly
age_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34","35-39","40+"]

# Place the data series into a new column inside of the DataFrame
purchase_df["Total Count"] = pd.cut(purchase_df["Age"], bins, labels= age_labels)


In [655]:
# Place the data series into a new column "Age Range" and place it in the dropped duplicate DataFrame
drop_dups["Age Range"] = pd.cut(drop_dups["Age"], bins, labels= age_labels)

# Calculate age count 
age_count = drop_dups["Age Range"].value_counts()

# Calculate percentages for each age labels
average_age = age_count / total_players * 100

# Create a data frame to display results in a tabular format
age_df = pd.DataFrame({"Total Count":age_count, 
                       "Percent of Players":average_age})

age_df = age_df.sort_index()

age_df["Percent of Players"] = age_df["Percent of Players"].map("{:,.2f}%".format)
age_df

C:\Users\cache\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Total Count,Percent of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame

